# CWT & STFT + CNN Model with BCI_competitionIII dataset

In [14]:
import mne
import numpy as np
from mne.datasets import eegbci
from mne.channels import make_standard_montage
from scipy.io import loadmat

epochs = [0,0,0,0,0]

for j in range(1,6):
    mat_data = loadmat('D:\BCI_competitionIII\BCI_100Hz\S' + str(j) + '_data_set')

    channels = []
    for i in range(0,118):
        channels.append(mat_data['nfo']['clab'][0][0][0][i][0])

    fs = mat_data['nfo']['fs'][0][0][0][0]
    events_index = mat_data['mrk']['pos'][0][0][0]

    events_labels = mat_data['mrk']['y'][0][0][0]
    events_labels= np.nan_to_num(events_labels, nan=0.0)

    info = mne.create_info(
        ch_names= channels,
        ch_types= ['eeg']*len(channels),
        sfreq= fs
    )

    info.set_montage('standard_1005', on_missing= 'ignore')
    ev = [i*fs*5 for i in range(len(events_labels))]
    events = np.column_stack((np.array(ev, dtype = int),
                            np.zeros(len(events_labels), dtype = int),
                            np.array(events_labels, dtype = int)))

    cnt = mat_data['cnt'] *0.1
    cnt = cnt.T

    data = []
    for i in range(0,len(events_labels)):
        if i <= 278:
            # print(np.shape(cnt[:,events_index[i]:events_index[i+1]]))
            data.append(cnt[:,events_index[i]:events_index[i] + (fs*5)])
        else:
            # print(np.shape(cnt[:,events_index[i]:len(cnt[0])]))
            data.append(cnt[:,events_index[i]:events_index[i]+ (fs*5)])

    data = np.asanyarray(data)

    t_min = 0.0
    t_max = 3.5
    event_id = dict(Right= 1, Foot= 2, no=0)
    epochs[j-1] = mne.EpochsArray(data, info, events,t_min, event_id, baseline=None)
    epochs[j-1] = epochs[j-1].copy().crop(tmin=0, tmax=2.5)
    epochs[j-1] = epochs[j-1].pick_channels(['C3','Cz','C4'])
    # epochs[j-1] = epochs[j-1].copy().filter(l_freq=0.5, h_freq=49.0, method = 'iir', iir_params= {"order": 12, "ftype":'butter'})
    # epochs[j-1] = epochs[j-1].pick_channels(['FC3','C5','CP3','C3','Cz','C4','C6','FC4','CP4'])
# eeg1 = eeg1.copy().set_eeg_reference(ref_channels="average")

Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated


In [15]:
combine_epochs = mne.concatenate_epochs([epochs[0], epochs[1], epochs[2], epochs[3], epochs[4]])
# combine_epochs = mne.concatenate_epochs([epochs[1]])
del epochs

Not setting metadata
1400 matching events found
No baseline correction applied


In [16]:
print(np.shape(combine_epochs['Right','Foot']))
labels = combine_epochs['Right','Foot'].events[:,2]

(560, 3, 351)


In [17]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import History 
history = History()
import pywt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from ssqueezepy import ssq_cwt, ssq_stft

In [18]:
combine_epochs['Right','Foot'].get_data().shape[2]

351

In [19]:
sfreq = 100
w_length = int(sfreq * 0.5)  # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, combine_epochs['Right','Foot'].get_data().shape[2] - w_length, w_step)

In [20]:
w_start

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300])

In [21]:
shape = np.shape(combine_epochs['Right','Foot'].get_data())

train_data = np.ndarray(shape=(shape[0], shape[2], shape[1]))
for jj in range(0, shape[0]):
    train_data[jj] = combine_epochs['Right','Foot'].get_data()[jj].T
print(np.shape(train_data))

(560, 351, 3)


In [22]:
# train_data = combine_epochs['Right','Foot'].get_data().reshape(shape[0], shape[2], shape[1])
print(np.shape(train_data))
scales = range(1,228)

scales = np.array(scales)
scales = scales.astype('float32')

waveletname = 'morl'
train_size = len(labels)
train_data_cwt = np.ndarray(shape=(train_size, 227, 227, 3))

for ii in range(0,train_size):
    if ii % 40 == 0:
        print(ii)
    for jj in range(0,3):
        signal = train_data[ii, :, jj]
        _ ,coeff, *_ = ssq_cwt(signal, scales = scales)
        # _ ,coeff, *_ = ssq_stft(signal, n_fft = 226*2)
        # coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:227]
        train_data_cwt[ii, :, :, jj] = coeff_
print(np.shape(coeff),np.shape(coeff_))

(560, 351, 3)
0


C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\494190546.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  train_data_cwt[ii, :, :, jj] = coeff_


40
80
120
160
200
240
280
320
360
400
440
480
520
(227, 351) (227, 227)


In [23]:
print(np.shape(coeff), np.shape(coeff_))

(227, 351) (227, 227)


In [24]:
print(np.shape(signal))
print(np.shape(coeff))
print(np.shape(coeff_))
print(np.shape(train_data_cwt))

(351,)
(227, 351)
(227, 227)
(560, 227, 227, 3)


In [25]:
x_train, x_test, y_train, y_test = train_test_split(train_data_cwt, labels, test_size=0.2, random_state=42)

img_x = 227
img_y = 227
img_z = 3
num_classes = 2

batch_size = 16
epochs = 30
print(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

# del train_data_cwt
# del combine_epochs
# plt.imshow(x_train[0,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
# plt.imshow(x_train[2,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
y_train = keras.utils.to_categorical(y_train - 1, num_classes)
y_test = keras.utils.to_categorical(y_test -1 , num_classes)
input_shape = (img_x, img_y, img_z)

(448, 227, 227, 3) (112, 227, 227, 3) (448,) (112,)


In [26]:
model = Sequential()
# Layer 1: Convolutional + Max-Pooling
model.add(Conv2D(filters= 96, kernel_size= (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 2: Convolutional + Max-Pooling
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1) ,activation='relu', padding= 'valid'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding= 'valid'))

# Layer 3: Convolutional
model.add(Conv2D(filters =384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# Layer 4: Convolutional
model.add(Conv2D(filters = 384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# Layer 5: Convolutional + Max-Pooling
model.add(Conv2D(filters = 256, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='valid'))

# Flatten the output for fully connected layers
model.add(Flatten())

# Layer 6: Fully Connected (Dense)
model.add(Dense(4048, activation='relu', input_shape = (13,13,128)))
model.add(keras.layers.Dropout(0.4))

# Layer 7: Fully Connected (Dense)
model.add(Dense(2048, activation='relu'))
model.add(keras.layers.Dropout(0.4))

# Layer 8: Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(), 
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, 
          epochs=epochs, verbose=1, 
          validation_data=(x_test, y_test), 
          callbacks=[history])

Epoch 1/30
28/28 [==============================] - 16s 523ms/step - loss: 1.9609 - accuracy: 0.4888 - val_loss: 0.6746 - val_accuracy: 0.5804
Epoch 2/30
28/28 [==============================] - 14s 495ms/step - loss: 0.6958 - accuracy: 0.5156 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 3/30
28/28 [==============================] - 14s 496ms/step - loss: 0.6944 - accuracy: 0.5692 - val_loss: 0.6963 - val_accuracy: 0.4911
Epoch 4/30
28/28 [==============================] - 14s 500ms/step - loss: 0.6953 - accuracy: 0.5067 - val_loss: 0.7001 - val_accuracy: 0.4911
Epoch 5/30
28/28 [==============================] - 14s 497ms/step - loss: 0.6932 - accuracy: 0.5179 - val_loss: 0.6933 - val_accuracy: 0.4911
Epoch 6/30
28/28 [==============================] - 14s 489ms/step - loss: 0.6939 - accuracy: 0.4866 - val_loss: 0.6934 - val_accuracy: 0.4911
Epoch 7/30
28/28 [==============================] - 14s 489ms/step - loss: 0.6943 - accuracy: 0.4643 - val_loss: 0.6933 - val_accuracy: 0.4911

KeyboardInterrupt: 

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Train loss: 0.6927924752235413, Train accuracy: 0.5139665007591248
Test loss: 0.6928182244300842, Test accuracy: 0.5111111402511597
